In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rfmpracticedata/rfm_data(1).csv


In [2]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

data = pd.read_csv("/kaggle/input/rfmpracticedata/rfm_data(1).csv")
print(data.head())

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location  
0     Tokyo  
1    London  
2  New York  
3    London  
4     Paris  


In [4]:
import pandas as pd
import numpy as np

In [8]:


# Convert the transaction date column to datetime format
data['PurchaseDate'] = pd.to_datetime(data['PurchaseDate'])

In [11]:
# Calculate recency by subtracting the latest transaction date from each customer's transaction date
data['recency'] = pd.to_datetime('2023-06-10') - data['PurchaseDate']


# Calculate Frequency
frequency_data = data.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID': 'Frequency'}, inplace=True)
data = data.merge(frequency_data, on='CustomerID', how='left')

# Calculate Monetary Value
monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)
data = data.merge(monetary_data, on='CustomerID', how='left')

In [12]:
print(data.head())

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location recency  Frequency  MonetaryValue  
0     Tokyo 60 days          1         943.31  
1    London 60 days          1         463.70  
2  New York 60 days          1          80.28  
3    London 60 days          1         221.29  
4     Paris 60 days          1         739.56  


In [20]:
# Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

In [23]:
# Calculate RFM scores
data['RecencyScore'] = pd.cut(data['recency'], bins=5, labels=recency_scores)
data['FrequencyScore'] = pd.cut(data['Frequency'], bins=5, labels=frequency_scores)
data['MonetaryScore'] = pd.cut(data['MonetaryValue'], bins=5, labels=monetary_scores)

In [24]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   CustomerID          1000 non-null   int64          
 1   PurchaseDate        1000 non-null   datetime64[ns] 
 2   TransactionAmount   1000 non-null   float64        
 3   ProductInformation  1000 non-null   object         
 4   OrderID             1000 non-null   int64          
 5   Location            1000 non-null   object         
 6   recency             1000 non-null   timedelta64[ns]
 7   Frequency           1000 non-null   int64          
 8   MonetaryValue       1000 non-null   float64        
 9   RecencyScore        1000 non-null   category       
 10  FrequencyScore      1000 non-null   category       
 11  MonetaryScore       1000 non-null   category       
dtypes: category(3), datetime64[ns](1), float64(2), int64(3), object(2), timedelta64[ns](1)
memo

In [25]:
# Convert RFM scores to numeric type
data['RecencyScore'] = data['RecencyScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)

In [26]:
# Calculate RFM score by combining the individual scores
data['RFM_Score'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']

# Create RFM segments based on the RFM score
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
data['Value Segment'] = pd.qcut(data['RFM_Score'], q=3, labels=segment_labels)

In [27]:
print(data.head())

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location recency  Frequency  MonetaryValue  RecencyScore  FrequencyScore  \
0     Tokyo 60 days          1         943.31             1               1   
1    London 60 days          1         463.70             1               1   
2  New York 60 days          1          80.28             1               1   
3    London 60 days          1         221.29             1               1   
4     Paris 60 days          1         739.56             1               1   

   MonetaryScore  RFM_Score Value Segment  
0              2  